In [1]:
import sys
sys.path.append("../")

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import math
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import plot
from tqdm import tqdm
from tabulate import tabulate
import pickle as pkl
pd.set_option("display.max_columns", None)

In [3]:
class Data:
    
    def __init__(self, path):
        self.df = {
            'raw': pd.read_pickle(path)
        }
        if 'time' in self.df['raw'].columns:
            self.df['raw']['time'] = [ x.replace(tzinfo=None) for x in self.df['raw']['time']]

    def __repr__(self) -> str:
        repr = str()
        for name, df in self.df.items():
            repr = repr + name + ':\n' + str(df.head(3)) + '\n'
        return repr

    def shorten(self, name: str, rows: int, direction: int, source: str='raw', cols: list=None):
        '''Create new dataframe with specified list of columns and number of rows
        direction: 1 if data should be selected from top and -1 if from bottom
        '''
        assert (direction != 1 or direction != -1), 'direction must be 1 (top) or -1 (bottom)'
        
        if cols == None:
            cols = self.df[source].columns
        if direction == 1:
            self.df[name] = self.df[source][cols].iloc[:rows].copy()
        else:
            self.df[name] = self.df[source][cols].iloc[-rows:].copy()
        self.df[name].reset_index(drop=True, inplace=True)

    def add_columns(self, name: str, cols: list):
        '''Add new columns to component dataframes
        '''        
        exist_cols = list(self.df[name].columns)
        cols = exist_cols + cols
        self.df[name] = self.df[name].reindex(columns = cols) 

    def prepare_fast_data(self, name: str):
        '''Prepare data as an array for fast processing
        fcols = {col1: col1_index, col2: col2_index, .... }     
        fdata = [array[col1], array[col2], array[col3], .... ]
        Accessed by: self.fdata[fcols[column_name]] for whole column or
                     self.fdata[fcols[column_name]][row_index] for a specific row item
        '''
        self.fcols = dict()
        for i in range(len(self.df[name].columns)):
            self.fcols[self.df[name].columns[i]] = i
        self.fastdf = [self.df[name][col].array for col in self.df[name].columns]

    def fdata(self, column: str, index: int=-1):
        assert index >= -1, 'Row index cannot be negative'
        if index == -1:
            return self.fastdf[self.fcols[column]]
        else:
            return self.fastdf[self.fcols[column]][index]
        
    def update_fdata(self, column: str, value, index: int=-1):
        assert index >= -1, 'Row index cannot be negative'
        if index == -1:
            self.fastdf[self.fcols[column]] = value
        else:
            self.fastdf[self.fcols[column]][index] = value


In [4]:
PATH = 'D:/Trading/forex_bot/outputs/'

In [5]:
# files = ['inputs.2.1.pkl', 'inputs.2.2.pkl', 'inputs.2.3.pkl']

In [6]:
# dfs = list()
# for f in files:
#     dfs.append(pd.read_pickle(PATH + f))

In [7]:
# inputs = pd.concat(dfs)
# inputs.drop_duplicates(inplace=True)
# inputs.reset_index(drop=True,inplace=True)
# inputs

In [8]:
# inputs.to_pickle(PATH + 'inputs.2.all.pkl')

In [9]:
d = Data(PATH + "inputs.all.pkl")

In [10]:
subset = ['init_signal', 'cushion', 'risk', 'rr', 'margin_closeout', 'streak_limit']
d.df['raw'].drop_duplicates(subset=subset, keep='first', inplace=True)
d.df['raw'].reset_index(drop=True, inplace=True)

In [11]:
cols = ['return_%', 'final_ac_bal', 'total_trades', 'avg_trade', 'avg_trade_duration', 'win_%', 'loss_%', 
        'expectancy', 'sharpe_ratio', 'sortino_ratio', 'calmar_ratio',
        'max_drawdown', 'max_ac_bal', 'cagr', 'min_ac_bal', 'max_margin_used', 'total_streaks', 'avg_trades_per_streak',
        'total_wins', 'max_win', 'avg_win', 'avg_win_duration', 'total_losses', 'max_loss', 'avg_loss', 'avg_loss_duration',
        'total_long', 'avg_long', 'avg_long_duration',
        'total_long_wins', 'avg_long_win', 'avg_long_win_duration',
        'total_long_losses', 'avg_long_loss', 'avg_long_loss_duration',
        'total_short', 'avg_short', 'avg_short_duration',
        'total_short_wins', 'avg_short_win', 'avg_short_win_duration',
        'total_short_losses', 'avg_short_loss', 'avg_short_loss_duration',
        
        ]
d.add_columns('raw', cols)
d.df['raw'].head(3)

,sim_name,init_signal,cushion,risk,rr,margin_closeout,streak_limit,return_%,final_ac_bal,total_trades,avg_trade,avg_trade_duration,win_%,loss_%,expectancy,sharpe_ratio,sortino_ratio,calmar_ratio,max_drawdown,max_ac_bal,cagr,min_ac_bal,max_margin_used,total_streaks,avg_trades_per_streak,total_wins,max_win,avg_win,avg_win_duration,total_losses,max_loss,avg_loss,avg_loss_duration,total_long,avg_long,avg_long_duration,total_long_wins,avg_long_win,avg_long_win_duration,total_long_losses,avg_long_loss,avg_long_loss_duration,total_short,avg_short,avg_short_duration,total_short_wins,avg_short_win,avg_short_win_duration,total_short_losses,avg_short_loss,avg_short_loss_duration
0,sim_1,1,1.5,0.001,1.5,True,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,sim_2,1,1.5,0.001,1.5,True,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,sim_3,-1,1.5,0.001,1.5,True,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
d.prepare_fast_data('raw')

In [13]:
d.fdata('sim_name', 0)

'sim_1'

In [14]:
def convert2date(date_time):
    return date_time.date().strftime("%Y-%m-%d")

In [15]:
def sharpe_ratio(returns: pd.Series, risk_free_rate: float=0) -> float:
    return (returns.mean() - risk_free_rate) / returns.std()

def sortino_ratio(returns: pd.Series, risk_free_rate: float=0, target_return: float=0) -> float:
    downside_returns = np.array([ret - risk_free_rate for ret in returns if ret < target_return])
    std_dev_downside = np.std(downside_returns, ddof=1) if len(downside_returns) > 0 else 0
    return (np.mean(returns) - risk_free_rate) / std_dev_downside if std_dev_downside != 0 else np.nan

def max_drawdown(returns: pd.Series) -> float:
    '''
    cum_returns: Calculates the cumulative returns using np.cumprod(1 + returns).
    peaks: Finds the peaks in the cumulative returns using np.maximum.accumulate(cum_returns).
    drawdowns: Calculates the drawdowns for each point by subtracting the peaks from the cumulative returns and dividing by the peaks.
    max_drawdown: Identifies the maximum drawdown by finding the minimum drawdown value using np.min(drawdowns).'''
    cum_returns = np.cumprod(1 + returns.values)
    peaks = np.maximum.accumulate(cum_returns)
    drawdowns = (cum_returns - peaks) / peaks
    max_drawdown = np.min(drawdowns)
    return max_drawdown

def calmar_ratio(returns: pd.Series):
    max_drawdown_value = max_drawdown(returns)
    annual_return = np.mean(returns) * 311  # Assuming 311 trading days per year for forex. Its usually 252 for stocks
    return annual_return / abs(max_drawdown_value)

def cagr(ac_bal: pd.Series, years: float) -> float:
    return (ac_bal.iloc[-1] / ac_bal.iloc[0]) ** (1 / years) - 1

In [16]:
def populate_results(d: Data, i: int, df: pd.DataFrame):
    d.update_fdata('return_%', (df['ac_bal'].iloc[-1] - df['ac_bal'].iloc[0]) / df['ac_bal'].iloc[0], i)
    d.update_fdata('final_ac_bal', df['ac_bal'].iloc[-1], i)
    d.update_fdata('cagr', cagr(df['ac_bal'], 8), i)
    d.update_fdata('max_ac_bal', df['ac_bal'].max(), i)
    d.update_fdata('min_ac_bal', df['ac_bal'].min(), i)
    d.update_fdata('max_margin_used', df['margin_used'].max(), i)
    d.update_fdata('total_streaks', df['streak_no'].iloc[-1], i)
    d.update_fdata('avg_trades_per_streak', df.groupby('streak_no')['trade_no'].max().mean(), i)
    
    trades = df[df.signal != 0].copy()
    trades['return'] = trades['ac_bal'].shift(-1) - trades['ac_bal']
    trades['return_%'] = trades['ac_bal'].pct_change().shift(-1)
    trades['index'] = trades.index
    trades['duration'] = trades['index'].shift(-1) - trades['index']
    trades.dropna(inplace=True)
    # print(trades[['ac_bal', 'return', 'return_%']].head())

    d.update_fdata('sharpe_ratio', sharpe_ratio(trades['return_%']), i)
    d.update_fdata('sortino_ratio', sortino_ratio(trades['return_%']), i)
    d.update_fdata('calmar_ratio', calmar_ratio(trades['return_%']), i)
    d.update_fdata('max_drawdown', max_drawdown(trades['return_%']), i)

    d.update_fdata('total_trades', trades.shape[0], i)
    d.update_fdata('avg_trade', trades['return'].mean(), i)
    d.update_fdata('avg_trade_duration', trades['duration'].mean(), i)
    
    wins = trades[trades['return'] > 0]
    d.update_fdata('total_wins', wins.shape[0], i)
    d.update_fdata('max_win', wins['return'].max(), i)
    d.update_fdata('avg_win', wins['return'].mean(), i)
    d.update_fdata('avg_win_duration', wins['duration'].mean(), i)
    d.update_fdata('win_%', d.fdata('total_wins', i) / d.fdata('total_trades', i), i)

    losses = trades[trades['return'] < 0]
    d.update_fdata('total_losses', losses.shape[0], i)
    d.update_fdata('max_loss', losses['return'].min(), i)
    d.update_fdata('avg_loss', losses['return'].mean(), i)
    d.update_fdata('avg_loss_duration', losses['duration'].mean(), i)
    d.update_fdata('loss_%', d.fdata('total_losses', i) / d.fdata('total_trades', i), i)
    
    longs = trades[trades.signal == 1]
    d.update_fdata('total_long', longs.shape[0], i)
    d.update_fdata('avg_long', longs['return'].mean(), i)
    d.update_fdata('avg_long_duration', longs['duration'].mean(), i)

    long_wins = longs[longs['return'] > 0]
    d.update_fdata('total_long_wins', long_wins.shape[0], i)
    d.update_fdata('avg_long_win', long_wins['return'].mean(), i)
    d.update_fdata('avg_long_win_duration', long_wins['duration'].mean(), i)

    long_losses = longs[longs['return'] < 0]
    d.update_fdata('total_long_losses', long_losses.shape[0], i)
    d.update_fdata('avg_long_loss', long_losses['return'].mean(), i)
    d.update_fdata('avg_long_loss_duration', long_losses['duration'].mean(), i)

    shorts = trades[trades.signal == -1]
    d.update_fdata('total_short', shorts.shape[0], i)
    d.update_fdata('avg_short', shorts['return'].mean(), i)
    d.update_fdata('avg_short_duration', shorts['duration'].mean(), i)

    short_wins = shorts[shorts['return'] > 0]
    d.update_fdata('total_short_wins', short_wins.shape[0], i)
    d.update_fdata('avg_short_win', short_wins['return'].mean(), i)
    d.update_fdata('avg_short_win_duration', short_wins['duration'].mean(), i)

    short_losses = shorts[shorts['return'] < 0]
    d.update_fdata('total_short_losses', short_losses.shape[0], i)
    d.update_fdata('avg_short_loss', short_losses['return'].mean(), i)
    d.update_fdata('avg_short_loss_duration', short_losses['duration'].mean(), i)


    d.update_fdata('expectancy', d.fdata('win_%', i) * d.fdata('avg_win', i) + d.fdata('loss_%', i) * d.fdata('avg_loss', i), i)

In [17]:
# def populate_results(d: Data, i: int, data: Data):
#     df = data['results'].copy()
#     d.update_fdata('return_%', (df['ac_bal'].iloc[-1] - df['ac_bal'].iloc[0]) / df['ac_bal'].iloc[0], i)
#     d.update_fdata('final_ac_bal', df['ac_bal'].iloc[-1], i)
#     d.update_fdata('cagr', cagr(df['ac_bal'], 8), i)
#     d.update_fdata('max_ac_bal', df['ac_bal'].max(), i)
#     d.update_fdata('min_ac_bal', df['ac_bal'].min(), i)
#     d.update_fdata('max_margin_used', df['margin_used'].max(), i)
#     d.update_fdata('total_streaks', df['streak_no'].iloc[-1], i)
#     d.update_fdata('avg_trades_per_streak', df.groupby('streak_no')['trade_no'].max().mean(), i)
    
#     trades = df[df.signal != 0].copy()
#     trades['return'] = trades['ac_bal'].shift(-1) - trades['ac_bal']
#     trades['return_%'] = trades['ac_bal'].pct_change().shift(-1)
#     trades['index'] = trades.index
#     trades['duration'] = trades['index'].shift(-1) - trades['index']
#     trades.dropna(inplace=True)
#     # print(trades[['ac_bal', 'return', 'return_%']].head())

#     d.update_fdata('sharpe_ratio', sharpe_ratio(trades['return_%']), i)
#     d.update_fdata('sortino_ratio', sortino_ratio(trades['return_%']), i)
#     d.update_fdata('calmar_ratio', calmar_ratio(trades['return_%']), i)
#     d.update_fdata('max_drawdown', max_drawdown(trades['return_%']), i)

#     d.update_fdata('total_trades', trades.shape[0], i)
#     d.update_fdata('avg_trade', trades['return'].mean(), i)
#     d.update_fdata('avg_trade_duration', trades['duration'].mean(), i)
    
#     wins = trades[trades['return'] > 0]
#     d.update_fdata('total_wins', wins.shape[0], i)
#     d.update_fdata('max_win', wins['return'].max(), i)
#     d.update_fdata('avg_win', wins['return'].mean(), i)
#     d.update_fdata('avg_win_duration', wins['duration'].mean(), i)
#     d.update_fdata('win_%', d.fdata('total_wins', i) / d.fdata('total_trades', i), i)

#     losses = trades[trades['return'] < 0]
#     d.update_fdata('total_losses', losses.shape[0], i)
#     d.update_fdata('max_loss', losses['return'].min(), i)
#     d.update_fdata('avg_loss', losses['return'].mean(), i)
#     d.update_fdata('avg_loss_duration', losses['duration'].mean(), i)
#     d.update_fdata('loss_%', d.fdata('total_losses', i) / d.fdata('total_trades', i), i)
    
#     longs = trades[trades.signal == 1]
#     d.update_fdata('total_long', longs.shape[0], i)
#     d.update_fdata('avg_long', longs['return'].mean(), i)
#     d.update_fdata('avg_long_duration', longs['duration'].mean(), i)

#     long_wins = longs[longs['return'] > 0]
#     d.update_fdata('total_long_wins', long_wins.shape[0], i)
#     d.update_fdata('avg_long_win', long_wins['return'].mean(), i)
#     d.update_fdata('avg_long_win_duration', long_wins['duration'].mean(), i)

#     long_losses = longs[longs['return'] < 0]
#     d.update_fdata('total_long_losses', long_losses.shape[0], i)
#     d.update_fdata('avg_long_loss', long_losses['return'].mean(), i)
#     d.update_fdata('avg_long_loss_duration', long_losses['duration'].mean(), i)

#     shorts = trades[trades.signal == -1]
#     d.update_fdata('total_short', shorts.shape[0], i)
#     d.update_fdata('avg_short', shorts['return'].mean(), i)
#     d.update_fdata('avg_short_duration', shorts['duration'].mean(), i)

#     short_wins = shorts[shorts['return'] > 0]
#     d.update_fdata('total_short_wins', short_wins.shape[0], i)
#     d.update_fdata('avg_short_win', short_wins['return'].mean(), i)
#     d.update_fdata('avg_short_win_duration', short_wins['duration'].mean(), i)

#     short_losses = shorts[shorts['return'] < 0]
#     d.update_fdata('total_short_losses', short_losses.shape[0], i)
#     d.update_fdata('avg_short_loss', short_losses['return'].mean(), i)
#     d.update_fdata('avg_short_loss_duration', short_losses['duration'].mean(), i)


#     d.update_fdata('expectancy', d.fdata('win_%', i) * d.fdata('avg_win', i) + d.fdata('loss_%', i) * d.fdata('avg_loss', i), i)

In [18]:
sims = d.df['raw'].shape[0]
for i in tqdm(range(sims), desc=" Analysing... "):
# for i in tqdm(range(2), desc=" Analysing... "):
    data = pd.read_pickle(PATH + d.fdata('sim_name', i) + '.pkl')
    assert (d.fdata('sim_name', i) == data['sim_name'] and
            d.fdata('init_signal', i) == data['init_signal'] and
            d.fdata('cushion', i) == data['cushion'] and
            d.fdata('risk', i) == data['risk'] and
            d.fdata('rr', i) == data['rr'] and
            d.fdata('margin_closeout', i) == data['margin_closeout'] and
            d.fdata('streak_limit', i) == data['streak_limit']), f"Parameters mismatch for {d.fdata('sim_name', i)}"
    
    populate_results(d, i, data['results'])

 Analysing... :   0%|          | 0/540 [00:00<?, ?it/s]C:\Users\souvi\AppData\Local\Temp\ipykernel_9352\2254183282.py:27: RuntimeWarning:

invalid value encountered in scalar power

 Analysing... :   0%|          | 1/540 [00:00<01:21,  6.64it/s]C:\Users\souvi\AppData\Local\Temp\ipykernel_9352\2254183282.py:27: RuntimeWarning:

invalid value encountered in scalar power

 Analysing... :   0%|          | 2/540 [00:00<01:17,  6.97it/s]C:\Users\souvi\AppData\Local\Temp\ipykernel_9352\2254183282.py:27: RuntimeWarning:

invalid value encountered in scalar power

 Analysing... :   1%|          | 3/540 [00:00<01:12,  7.44it/s]C:\Users\souvi\AppData\Local\Temp\ipykernel_9352\2254183282.py:27: RuntimeWarning:

invalid value encountered in scalar power

C:\Users\souvi\AppData\Local\Temp\ipykernel_9352\2254183282.py:27: RuntimeWarning:

invalid value encountered in scalar power

 Analysing... :   1%|          | 5/540 [00:00<00:58,  9.09it/s]C:\Users\souvi\AppData\Local\Temp\ipykernel_9352\225418328

KeyboardInterrupt: 

In [ ]:
data

In [ ]:
with open(PATH + 'hedging_sim_analysis.2.pkl', 'wb') as file:
    pkl.dump(d.df['raw'], file)

In [ ]:
d.df['raw'].head()

In [ ]:
with open(PATH + 'hedging_sim_analysis.2.pkl', 'rb') as f:
    df_all = pkl.load(f)

In [ ]:
df_all.drop_duplicates(subset=['init_signal', 'cushion', 'risk', 'rr', 'margin_closeout', 'streak_limit'],keep='first')
df_all

In [ ]:
df = df_all[df_all.margin_closeout == True].copy()
df.reset_index(inplace=True)
df

In [ ]:
df0 = df_all[df_all.margin_closeout == False].copy()
df0.reset_index(inplace=True)
df0

In [ ]:
# figs1 = [
#             [px.scatter(df, x="cushion", y="return_%", color="init_signal", symbol="streak_limit"),
#             px.scatter(df, x="cushion", y="expectancy", color="init_signal", symbol="streak_limit")],
#             [px.scatter(df, x="cushion", y="win_%", color="init_signal", symbol="streak_limit"),
#             px.scatter(df, x="cushion", y="loss_%", color="init_signal", symbol="streak_limit")]
#     ]

# fig = make_subplots(
#     rows=2, cols=2,
#     subplot_titles=("return_%", "expectancy", "win_%", "loss_%"))

# for i, f1 in enumerate(figs1):
#     for j, f2 in enumerate(f1):
#         for trace in range(len(f2["data"])):
#             fig.append_trace(f2["data"][trace], row=i+1, col=j+1)
        
# fig.show()

In [ ]:
px.scatter(df, x = 'rr', y = 'win_%', color = 'risk')

In [ ]:
# win_% decreases with rr, lower rr higher win_%

In [ ]:
px.scatter(df, x = 'risk', y = 'win_%', color='rr')

In [ ]:
# win_% increases with risk if rr <= 2, higher risk higher win_% if rr <= 2

In [ ]:
px.scatter(df, x = 'cushion', y = 'expectancy', color = 'rr', facet_col='risk')

In [ ]:
px.scatter(df, x = 'cushion', y = 'expectancy', color = 'risk', facet_col='streak_limit')

In [ ]:
# higher cushion, higher expectancy if risk >= 0.002

In [ ]:
px.scatter(df, x = 'risk', y = 'expectancy', color = 'streak_limit', facet_col='streak_limit')

In [ ]:
px.scatter(df, x = 'cushion', y = 'expectancy', color = 'streak_limit', facet_col='risk')

In [ ]:
px.scatter(df, x = 'risk', y = 'max_loss')

In [ ]:
# max loss decreases with risk, higher risk lower max loss

In [ ]:
px.scatter(df, x='rr', y='max_loss', facet_col='streak_limit')

In [ ]:
# max loss increases with lower rr but only for higher streak limits

In [ ]:
px.scatter(df, x = 'risk', y = 'max_loss', color = 'streak_limit', facet_col='rr')

In [ ]:
# max_loss decreases with risk, higher risk lower max loss

In [ ]:
def plot_plots(df):  
    px.scatter(df, x = 'streak_limit', y = 'expectancy', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'streak_limit', y = 'return_%', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'streak_limit', y = 'max_margin_used', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'streak_limit', y = 'min_ac_bal', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'streak_limit', y = 'final_ac_bal', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'streak_limit', y = 'max_loss', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'streak_limit', y = 'max_win', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'streak_limit', y = 'sharpe_ratio', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'streak_limit', y = 'sortino_ratio', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'streak_limit', y = 'calmar_ratio', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'streak_limit', y = 'max_drawdown', color = 'risk', facet_col='rr').show()

In [ ]:
df1 = df[(df.risk>=0.003) & (df.rr <= 2) & (df.cushion<=3)]
df1

In [ ]:
plot_plots(df1)

In [ ]:
def plot_plots(df):  
    px.scatter(df, x = 'streak_limit', y = 'expectancy', color = 'cushion', facet_col='rr').show()
    px.scatter(df, x = 'streak_limit', y = 'expectancy', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'streak_limit', y = 'return_%', color = 'cushion', facet_col='rr').show()
    px.scatter(df, x = 'streak_limit', y = 'return_%', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'streak_limit', y = 'max_margin_used', color = 'cushion', facet_col='rr').show()
    px.scatter(df, x = 'streak_limit', y = 'max_margin_used', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'streak_limit', y = 'min_ac_bal', color = 'cushion', facet_col='rr').show()
    px.scatter(df, x = 'streak_limit', y = 'min_ac_bal', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'streak_limit', y = 'final_ac_bal', color = 'cushion', facet_col='rr').show()
    px.scatter(df, x = 'streak_limit', y = 'final_ac_bal', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'streak_limit', y = 'max_loss', color = 'cushion', facet_col='rr').show()
    px.scatter(df, x = 'streak_limit', y = 'max_loss', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'streak_limit', y = 'max_win', color = 'cushion', facet_col='rr').show()
    px.scatter(df, x = 'streak_limit', y = 'max_win', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'streak_limit', y = 'sharpe_ratio', color = 'cushion', facet_col='rr').show()
    px.scatter(df, x = 'streak_limit', y = 'sharpe_ratio', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'streak_limit', y = 'sortino_ratio', color = 'cushion', facet_col='rr').show()
    px.scatter(df, x = 'streak_limit', y = 'sortino_ratio', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'streak_limit', y = 'calmar_ratio', color = 'cushion', facet_col='rr').show()
    px.scatter(df, x = 'streak_limit', y = 'calmar_ratio', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'streak_limit', y = 'max_drawdown', color = 'cushion', facet_col='rr').show()
    px.scatter(df, x = 'streak_limit', y = 'max_drawdown', color = 'risk', facet_col='rr').show()

In [ ]:
plot_plots(df1)

In [ ]:
df2 = df1[df1.rr==1.5].copy()
df2.head()

In [ ]:
def plot_plots(df):  
    px.scatter(df, x = 'streak_limit', y = 'expectancy', color = 'cushion', facet_col='risk').show()
    px.scatter(df, x = 'streak_limit', y = 'return_%', color = 'cushion', facet_col='risk').show()
    px.scatter(df, x = 'streak_limit', y = 'cagr', color = 'cushion', facet_col='risk').show()
    px.scatter(df, x = 'streak_limit', y = 'max_margin_used', color = 'cushion', facet_col='risk').show()
    px.scatter(df, x = 'streak_limit', y = 'min_ac_bal', color = 'cushion', facet_col='risk').show()
    px.scatter(df, x = 'streak_limit', y = 'final_ac_bal', color = 'cushion', facet_col='risk').show()
    px.scatter(df, x = 'streak_limit', y = 'max_loss', color = 'cushion', facet_col='risk').show()
    px.scatter(df, x = 'streak_limit', y = 'max_win', color = 'cushion', facet_col='risk').show()
    px.scatter(df, x = 'streak_limit', y = 'sharpe_ratio', color = 'cushion', facet_col='risk').show()
    px.scatter(df, x = 'streak_limit', y = 'sortino_ratio', color = 'cushion', facet_col='risk').show()
    px.scatter(df, x = 'streak_limit', y = 'calmar_ratio', color = 'cushion', facet_col='risk').show()
    px.scatter(df, x = 'streak_limit', y = 'max_drawdown', color = 'cushion', facet_col='risk').show()

In [ ]:
plot_plots(df2)

In [ ]:
df_final = df[(df.rr == 1.5) & (df.risk == 0.004) & (df.cushion == 2) & (df.streak_limit == 5)]
df_final

In [ ]:
40 * 10 ** -4

win_% decreases with rr, lower rr higher win_%

win_% increases with risk, higher risk higher win_%

expectancy increases with risk, higher risk higher expectancy

init_signal has not effect on win_%


In [ ]:
df1.sort_values(by='final_ac_bal')

In [ ]:
df.calmar_ratio.max()

In [ ]:
with open(PATH + 'sim_2250.pkl', 'rb') as f:
    tempdf = pkl.load(f)
tempdf = tempdf['results'].copy()

In [ ]:
trades = tempdf[tempdf.signal!=0].copy()
trades.head()

In [ ]:
trades.ac_bal.pct_change().shift(-1)
px.histogram(x=trades.ac_bal.pct_change().shift(-1))

In [ ]:
returns = trades['ac_bal'].shift(-1) - trades['ac_bal']
px.histogram(x=returns)

In [ ]:
df.sort_values(by='final_ac_bal')